In [349]:
# import libraries
# DataFrame
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# sklearny
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB


#tensorflow
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Utility
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import string
import pickle
# RNN - LSTM
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re
import keras
from keras import regularizers
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Test Input dataset before model training

In [350]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

{'all', 'needn', 'because', 'up', "she'll", 'or', "you've", "shan't", 'over', 'where', 'wasn', 'into', 'out', "i'll", 'that', 'yourself', 'by', 'same', 'at', 'before', 'hasn', 'll', 'mustn', 'don', 'when', 'it', 'hadn', 'very', "they'd", 'you', 'after', 'the', "you'd", 'their', 'those', "mightn't", 'which', 'only', 'what', 'i', 'has', 'will', 'can', 'each', 'she', 're', "that'll", "they've", 'no', 'this', "hasn't", "he'd", 'down', "needn't", 'isn', 'm', 'her', 'between', 'whom', 'such', 'is', 'just', 'hers', "hadn't", 'haven', "i'm", 'and', 'most', 'own', "they're", "i've", 'there', 'during', 'through', "he's", 'ours', 'doing', 'am', 'mightn', "it'll", 'we', 'how', 'itself', 'y', 'our', 'any', 'once', 'few', 'wouldn', 'are', 'd', 'these', 'as', 'ma', 'himself', 'both', "couldn't", 'having', 'me', 'than', "it's", 'o', "you'll", 'be', "doesn't", 'not', 'some', "we're", "won't", 't', 's', 'had', 'against', "i'd", 'aren', 'should', 'nor', 'have', 'further', 'with', 'a', 'too', "aren't", 't

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhaijingjing/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [351]:
negation_words = {"no", "nor", "not", "don", "don't", "didn", "didn't", "doesn", "doesn't", "can't", "cannot", "won't", "wouldn", "wouldn't", "shouldn", "shouldn't", "isn't", "aren't", "wasn", "wasn't", "weren't", "hadn", "hadn't", "", "wouldn't"}
custom_stop_words = stopword.difference(negation_words)
print(custom_stop_words)

{'all', 'needn', 'because', 'up', "she'll", 'or', "you've", "shan't", 'over', 'where', 'into', 'out', "i'll", 'that', 'yourself', 'by', 'same', 'at', 'before', 'hasn', 'll', 'mustn', 'when', 'it', 'very', "they'd", 'you', 'after', 'the', "you'd", 'their', 'those', "mightn't", 'which', 'only', 'what', 'i', 'has', 'will', 'can', 'each', 'she', 're', "that'll", "they've", 'this', "hasn't", "he'd", 'down', "needn't", 'isn', 'm', 'her', 'between', 'whom', 'such', 'is', 'just', 'hers', 'haven', "i'm", 'and', 'most', 'own', "they're", "i've", 'there', 'during', 'through', "he's", 'ours', 'doing', 'am', 'mightn', "it'll", 'we', 'how', 'itself', 'y', 'our', 'any', 'once', 'few', 'are', 'd', 'these', 'as', 'ma', 'himself', 'both', "couldn't", 'having', 'me', 'than', "it's", 'o', "you'll", 'be', 'some', "we're", 't', 's', 'had', 'against', "i'd", 'aren', 'should', 'have', 'further', 'with', 'a', 'too', 'them', 'were', 'ourselves', 'theirs', "we'll", 'but', 'off', 'for', 'they', 'themselves', 'was

In [352]:
{'all', 'needn', 'because', 'up', "she'll", 'or', "you've", "shan't", 'over', 'where', 'wasn', 'into', 'out', "i'll", 'that', 'yourself', 'by', 'same', 'at', 'before', 'hasn', 'll', 'mustn', 'don', 'when', 'it', 'hadn', 'very', "they'd", 'you', 'after', 'the', "you'd", 'their', 'those', "mightn't", 'which', 'only', 'what', 'i', 'has', 'will', 'can', 'each', 'she', 're', "that'll", "they've", 'no', 'this', "hasn't", "he'd", 'down', "needn't", 'isn', 'm', 'her', 'between', 'whom', 'such', 'is', 'just', 'hers', "hadn't", 'haven', "i'm", 'and', 'most', 'own', "they're", "i've", 'there', 'during', 'through', "he's", 'ours', 'doing', 'am', 'mightn', "it'll", 'we', 'how', 'itself', 'y', 'our', 'any', 'once', 'few', 'wouldn', 'are', 'd', 'these', 'as', 'ma', 'himself', 'both', "couldn't", 'having', 'me', 'than', "it's", 'o', "you'll", 'be', "doesn't", 'not', 'some', "we're", "won't", 't', 's', 'had', 'against', "i'd", 'aren', 'should', 'nor', 'have', 'further', 'with', 'a', 'too', "aren't", 'them', 'were', 'ourselves', 'theirs', "we'll", 'but', 'off', 'for', 'they', 'doesn', "wouldn't", 'themselves', 'was', 'yours', 'yourselves', 'in', 'about', 'then', 'until', 'again', 'shan', 'him', 'of', "we've", 'its', "didn't", 'myself', "should've", "isn't", 'while', 'your', 'why', 'so', 'an', "it'd", 'didn', 'on', "you're", 'above', 'couldn', "shouldn't", 'does', "haven't", 'if', 'being', 'his', 'from', 'other', 'more', "weren't", 'do', 'my', 'under', 'did', 'now', 'herself', "we'd", 'who', 'here', 've', 'shouldn', 'below', "she'd", 'been', "mustn't", "they'll", "wasn't", 'weren', 'won', 'he', "he'll", "don't", 'to', 'ain', "she's"}

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [353]:
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
punctuationPattern = r"[^\w\s']|(?<!\w)n\'t"
def process_tweets_not_del_stopw(tweet):
  # Lower Casing
    tweet = tweet.lower()
    # tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    # final_tokens = [w for w in tokens if w not in stopword]
    final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_negation_append_del_stopw(tweet):
    # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    # tweet = tweet.translate(str.maketrans("","",string.punctuation))
    tweet = re.sub(punctuationPattern,'',tweet)
    #tokenizing words
    tokens = word_tokenize(tweet)
    # Remove punctuations but remain ‘n't’
    final_tokens = tokens
    print(f"tokens: {tokens}")
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    negation_words = {"not", "no", "never", "n't"}
    prev_neg = False

    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        #finalwords.append(word)
        if word in negation_words:
          prev_neg = True
        elif prev_neg:
          finalwords.append("not_" + word)
          prev_neg = False
        else:
          finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_negation_append(tweet):
  # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    # tweet = tweet.translate(str.maketrans("","",string.punctuation))
    tweet = re.sub(punctuationPattern,'',tweet)
    #tokenizing words
    tokens = word_tokenize(tweet)
    # Remove punctuations but remain ‘n't’
    final_tokens = tokens
    print(f"tokens: {tokens}")
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    negation_words = {"not", "no", "never", "n't"}
    prev_neg = False

    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        #finalwords.append(word)
        if word in negation_words:
          prev_neg = True
        elif prev_neg:
          finalwords.append("not_" + word)
          prev_neg = False
        else:
          finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_remain_negation(tweet):
    # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    print(f"Remove punctuations: {tweet}")
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    negation_words = {"no", "nor", "not", "don", "don't", "didn", "didn't", "doesn", "doesn't", "can't", "cannot", "won't", "wouldn", "wouldn't", "shouldn", "shouldn't", "isn't", "aren't", "wasn't", "weren't"}

    # Remove negation words from the stop word list
    custom_stop_words = stopword.difference(negation_words)
    # print(f"custom_stop_words: {custom_stop_words}")

    final_tokens = [w for w in tokens if w not in custom_stop_words]
    print(f"tokens: {tokens}")
    #final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_del_stopw(tweet):
  # Lower Casing
    tweet = tweet.lower()
    # tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    print(f"tokens: {tokens}")
    #Removing Stop Words
    final_tokens = [w for w in tokens if w not in stopword]
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

In [354]:
def tokenizer(max_words, test_text):
    # 1. build word table with top 50 words
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(test_text)
    print("单词表:", tokenizer.word_index)

    sequences = tokenizer.texts_to_sequences(test_text)
    print("文本的单词索引序列:", sequences)

    tweets = pad_sequences(sequences, maxlen=10)
    print(tweets.shape)
    print("填充后文本的单词索引序列:", tweets)

In [355]:
# Test
max_words = 100
test_tweets = [
    "Aww... @Wowena just got home from church??? He doesn't go today",
    "You aren't happy with the service at all",
    "i'll never come back! website at https://www.example.com",
    "Not happy with the service at all."
]

processed_test_text = [process_tweets_negation_append_del_stopw(x) for x in test_tweets]
print("预处理后的数据:", processed_test_text)
tokenizer(max_words, processed_test_text)

tokens: ['aww', 'just', 'got', 'home', 'from', 'church', 'he', 'does', "n't", 'go', 'today']
tokens: ['you', 'are', "n't", 'happy', 'with', 'the', 'service', 'at', 'all']
tokens: ['i', "'ll", 'never', 'come', 'back', 'website', 'at']
tokens: ['not', 'happy', 'with', 'the', 'service', 'at', 'all']
预处理后的数据: ['aww just got home from church he doe not_go today', 'you are not_happy with the service at all', "'ll not_come back website at", 'not_happy with the service at all']
单词表: {'not': 1, 'at': 2, 'happy': 3, 'with': 4, 'the': 5, 'service': 6, 'all': 7, 'aww': 8, 'just': 9, 'got': 10, 'home': 11, 'from': 12, 'church': 13, 'he': 14, 'doe': 15, 'go': 16, 'today': 17, 'you': 18, 'are': 19, "'ll": 20, 'come': 21, 'back': 22, 'website': 23}
文本的单词索引序列: [[8, 9, 10, 11, 12, 13, 14, 15, 1, 16, 17], [18, 19, 1, 3, 4, 5, 6, 2, 7], [20, 1, 21, 22, 23, 2], [1, 3, 4, 5, 6, 2, 7]]
(4, 10)
填充后文本的单词索引序列: [[ 9 10 11 12 13 14 15  1 16 17]
 [ 0 18 19  1  3  4  5  6  2  7]
 [ 0  0  0  0 20  1 21 22 23  2]
 [ 